In [1]:
import os
from pinecone import Pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

c:\Users\Kusal\.conda\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [4]:
# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf("../data")

In [5]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 33118


In [7]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [9]:
# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

In [10]:
index_name = "mental-chatbot"

In [11]:

from langchain_pinecone import PineconeVectorStore

# Assuming LP is an alias for LangchainPinecone
docsearch = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    embeddings,
    index_name=index_name
)


In [12]:
# If we already have an index we can load it like this
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

In [13]:
query = "What is Anxiety"

docs = docsearch.similarity_search(query, k=3)
print("Result", docs)


Result [Document(page_content='a psychiatric disorder that is characterized by intense fear and anxiety, which result in\nbehavioral disturbances that can benefit from treatment. Specific anxiety disorders include\nagoraphobia, generalized anxiety disorder, social anxiety disorder, specific phobias, and\npanic disorder. Anxiety can also be a neuropsychiatric manifestation of a malignancy, a\nsomatic response to medical complications (eg, dyspnea), or can be induced by medication'), Document(page_content='(eg, career or family), fear of dependency upon others, and fear of closeness (eg, in paranoid\npatients). Patients may be aware of some fears but unaware of others.\nIn most diseases, some degree of anxiety is expected and adaptive because it puts the patient\non alert. However, even normal anxiety may occasionally have adverse effects. As an example,\nimmediately after an acute myocardial infarction, anxiety-associated increases in heart rate and'), Document(page_content='F. The fear

In [15]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}



In [16]:
llm = CTransformers(
    model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)



In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs={})


In [18]:

while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\Kusal\.conda\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :   There are several things you can do to help manage your anxiety symptoms. Here are a few suggestions:
1. Practice relaxation techniques such as deep breathing, progressive muscle relaxation, or visualization. These can help calm the body and mind.
2. Exercise regularly. Exercise has been shown to have a positive impact on anxiety symptoms. Find an activity that you enjoy and try to do it for at least 30 minutes a day.
3. Challenge negative thoughts. Cognitive-behavioral therapy (CBT) is a helpful approach in managing anxiety disorders by identifying and challenging negative thought patterns.
4. Seek support from friends, family, or a mental health professional. Talking to someone about your feelings can help you feel less alone and provide valuable insight into how to manage your symptoms.
5. Consider seeking professional treatment option - Take medications can try journaling for individuals who you should avoid stimulus Avoid drugs, Try mindfulfill out of the use positive

KeyboardInterrupt: 